# Feature Selection Using AIC

### Overview

Feature Selection using AIC algorithm

### Builds on
None

### Run time
approx. 20 minutes

### Notes


## Step 1 - Load Data

In [1]:
import os
import urllib.request

data_url = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
data_location = os.path.basename(data_url)

if not os.path.exists (data_location):
    print("Downloading : ", data_url)
    urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)

data_location: house-sales-full.csv


In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

house_prices = pd.read_csv(data_location)
house_prices.sample(10)

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
21656,21657,6/17/13,618000,7853301570,Single Family,6/1/13,366100,0.84,"734,645.00",1,...,2.50,4,9,2006,0,0,207000,435000,98065,False
21887,21888,2/17/10,840000,7856600170,Single Family,2/1/10,350900,0.81,"1,041,801.00",1,...,2.50,4,8,1968,2005,0,481000,380000,98006,False
14612,14613,3/16/07,419000,4307340050,Single Family,3/1/07,429600,0.99,"424,462.00",1,...,2.50,3,7,2004,0,0,154000,188000,98056,False
20928,20929,6/2/11,275000,7694600143,Single Family,6/1/11,315500,0.72,"379,334.00",1,...,1.75,3,7,1957,0,1,88000,239000,98146,False
10975,10976,4/12/06,449850,3336000626,Single Family,4/1/06,396500,0.91,"493,757.00",1,...,1.50,3,8,1909,0,0,106000,274000,98118,False
3107,3108,7/26/06,380000,1189000207,Townhouse,7/1/06,407700,0.94,"405,632.00",1,...,2.50,2,8,2001,0,2,84000,312000,98122,False
26978,26979,6/18/07,225000,9830200510,Single Family,6/1/07,433200,1.00,"226,039.00",1,...,1.00,2,7,1947,0,0,273000,197000,98118,False
256,257,2/16/06,325000,106000360,Single Family,2/1/06,387600,0.89,"364,912.00",1,...,1.00,2,6,1948,0,0,301000,56000,98177,False
685,686,12/5/10,466000,259800680,Single Family,12/1/10,330600,0.76,"613,440.00",1,...,2.25,4,7,1965,0,0,311000,272000,98008,False
17759,17760,7/26/06,673750,6381501280,Single Family,7/1/06,407700,0.94,"719,195.00",1,...,3.75,5,8,1955,0,0,196000,391000,98125,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [3]:
house_prices.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [4]:
house_prices.corr()

<ipython-input-4-61669d780222>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  house_prices.corr()


,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data
Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [5]:
input_columns= ['zhvi_px', 'zhvi_idx', 'NbrLivingUnits', 'SqFtLot', 'SqFtTotLiving', 'SqFtFinBasement', 'Bathrooms', 
                'Bedrooms', 'BldgGrade',   'YrBuilt', 'YrRenovated', 'TrafficNoise', 'LandVal', 'ImpsVal', 'ZipCode', 'NewConstruction']

label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice
0,405100,0.93,2,9373,2400,0,3.00,6,7,1991,0,0,70000,229000,98002,False,280000
1,404400,0.93,1,20156,3764,1452,3.75,4,10,2005,0,0,203000,590000,98166,True,1000000
2,425600,0.98,1,26036,2060,900,1.75,4,8,1947,0,0,183000,275000,98166,False,745000
3,418400,0.96,1,8618,3200,1640,3.75,5,7,1966,0,0,104000,229000,98168,False,425000
4,351600,0.81,1,8620,1720,0,1.75,4,7,1948,0,0,104000,205000,98168,False,240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27058,332400,0.76,1,1161,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,False,374000
27059,339400,0.78,1,1005,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,True,374000
27060,325300,0.75,1,11170,1070,0,1.00,4,6,1971,0,0,92000,130000,98055,False,165000
27061,400600,0.92,1,6223,1345,0,2.00,3,7,1939,0,0,103000,212000,98166,False,315000


In [6]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 16)
y.shape =  (27063, 1)


## Step 4 : Run Regression

In [7]:
# from sklearn.linear_model import LinearRegression
from sklearn.linear_model import  LassoLarsIC
import numpy as np


# algo + data --> model
# algo =  LinearRegression ()
algo = LassoLarsIC(criterion='aic', max_iter=500)

model = algo.fit(x, np.ravel(y))  # <--- training
model


LassoLarsIC()

## Step 5 : Predict

In [8]:
## TODO Predict on 'x'
predictions = model.predict(x)
predictions

array([322853.64309764, 818437.37464157, 496723.33993593, ...,
       125615.73534232, 319873.78072524, 556915.94517578])

In [9]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a.sample(10)

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice,predicted_price
2877,347800,0.80,1,6833,4460,1320,3.00,4,10,1955,2007,0,688000,1354000,98112,False,1675000,"1,632,685.01"
18784,429600,0.99,1,4646,2840,630,3.00,3,8,1949,2005,0,550000,389000,98119,False,985000,"933,417.84"
4641,334700,0.77,1,7110,1380,0,1.00,4,6,1939,0,0,122000,91000,98030,False,186000,"156,655.61"
4942,427800,0.98,1,7485,2460,1230,2.50,3,7,1946,2013,0,81000,266000,98168,False,389950,"380,642.07"
19862,311600,0.72,1,4800,2009,0,2.50,4,8,2012,0,0,83000,214000,-1,True,313147,"231,756.09"
759,396500,0.91,1,2201,1600,0,2.50,3,7,2003,0,0,149000,244000,98027,False,425000,"376,136.43"
9473,316900,0.73,1,5737,2380,0,2.50,4,7,2010,0,0,80000,271000,-1,True,298000,"265,585.51"
5003,423400,0.97,1,2623,2065,60,2.50,2,10,2006,0,1,171000,938000,98119,True,1100000,"967,607.21"
11707,370700,0.85,1,14130,4340,330,3.50,4,11,2005,0,0,770000,1363000,98004,False,2545000,"1,774,799.30"
15444,418300,0.96,1,5544,2000,0,2.50,4,8,2003,0,0,108000,256000,98038,False,392000,"394,472.84"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

**Q==> Also inspect R2 value.  Is it decent?**

In [10]:
## coefficients
import numpy as np
np.set_printoptions(precision=2, suppress=True)

print ("coefficients: " , model.coef_)
print ("intercept : ", model.intercept_)

coefficients:  [    1.16     0.    9155.4      0.37    34.9     -3.99  6945.5  -8461.43
 17659.24  -194.66    -9.58 -1174.17     0.94     0.61     0.33 43819.15]
intercept :  -194561.7372396295


In [11]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "coefficient": model.coef_})
coef

,input_column,coefficient
0,zhvi_px,1.16
1,zhvi_idx,0.00
2,NbrLivingUnits,"9,155.40"
3,SqFtLot,0.37
4,SqFtTotLiving,34.90
5,SqFtFinBasement,-3.99
6,Bathrooms,"6,945.50"
7,Bedrooms,"-8,461.43"
8,BldgGrade,"17,659.24"
9,YrBuilt,-194.66


In [12]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

## TODO : find R2 by comparing 'y' and 'predictions'
print ("R2 : " , r2_score(y, predictions))

print ("MSE : ", mean_squared_error(y, predictions))
print ("RMSE : ", sqrt(mean_squared_error(y, predictions)) )

R2 :  0.8424833143425134
MSE :  18511677673.415257
RMSE :  136057.62629641625
